In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.model_selection import train_test_split, KFold  # Fixing the import statement
import numpy as np
from imblearn.over_sampling import SMOTE

In [4]:
datafile_train=r'D:\NIDHI AI\P1 Data/Consumer_Complaints_train.csv'
datafile_test=r'D:\NIDHI AI\P1 Data/Consumer_Complaints_test_share.csv'
cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

In [5]:
#cd_train.head(5)

In [6]:
#cd_train['Product'].nunique()

In [7]:
#cd_train.dtypes

In [8]:
for col in ['Date received','Date sent to company']:
    cd_train[col]=pd.to_datetime(cd_train[col],infer_datetime_format=True)
    cd_test[col]=pd.to_datetime(cd_test[col],infer_datetime_format=True)

In [9]:
#cd_train

In [10]:
cd_train['day_diff']=pd.to_numeric(cd_train['Date sent to company']-cd_train['Date received'])
cd_test['day_diff']=pd.to_numeric(cd_test['Date sent to company']-cd_test['Date received'])

In [11]:
#cd_train

In [12]:
#cd_train['Consumer complaint narrative'].nunique()

In [13]:
for col in ['Date received','Date sent to company']:
    cd_train.drop([col],axis=1,inplace=True)
    cd_test.drop([col],axis=1,inplace=True)

In [14]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 74019
Company public response : 10
Company : 3276
State : 62
ZIP code : 25962
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2


In [15]:
cd_train.isnull().sum()

Product                              0
Sub-product                     138473
Issue                                0
Sub-issue                       292625
Consumer complaint narrative    403327
Company public response         388029
Company                              0
State                             3839
ZIP code                          3848
Tags                            411215
Consumer consent provided?      342934
Submitted via                        0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
day_diff                             0
dtype: int64

In [14]:
#len(pd.isnull(cd_train['Tags']))
#len(cd_train)

In [16]:
cd_train.shape

(478421, 17)

In [17]:
cd_test.shape

(119606, 16)

In [18]:
cd_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119606 entries, 0 to 119605
Data columns (total 16 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Product                       119606 non-null  object
 1   Sub-product                   84854 non-null   object
 2   Issue                         119606 non-null  object
 3   Sub-issue                     46546 non-null   object
 4   Consumer complaint narrative  18557 non-null   object
 5   Company public response       22776 non-null   object
 6   Company                       119606 non-null  object
 7   State                         118681 non-null  object
 8   ZIP code                      118680 non-null  object
 9   Tags                          16871 non-null   object
 10  Consumer consent provided?    33864 non-null   object
 11  Submitted via                 119605 non-null  object
 12  Company response to consumer  119606 non-null  object
 13 

In [19]:
for col in ['Sub-product','Sub-issue','Consumer complaint narrative',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_train[varname]=np.where(pd.isnull(cd_train[col]),1,0)
    cd_train.drop([col],axis=1,inplace=True)
    cd_test[varname]=np.where(pd.isnull(cd_test[col]),1,0)
    cd_test.drop([col],axis=1,inplace=True)
    

In [20]:
cd_train.head(4)

,Product,Issue,Company,State,ZIP code,Submitted via,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,day_diff,Sub_product_isNan,Sub_issue_isNan,Consumer_complaint_narrative_isNan,Company_public_response_isNan,Tags_isNan,Consumer_consent_provided_isNan
0,Credit card,Billing statement,Wells Fargo & Company,MI,48342,Web,Closed with explanation,Yes,No,856103,86400000000000,1,1,1,1,0,1
1,Bank account or service,"Making/receiving payments, sending money",Santander Bank US,PA,18042,Referral,Closed,Yes,No,1034666,518400000000000,0,1,1,1,1,1
2,Credit reporting,Incorrect information on credit report,Equifax,CA,92427,Referral,Closed with non-monetary relief,Yes,No,756363,1814400000000000,1,0,1,1,1,1
3,Credit card,Billing statement,U.S. Bancorp,GA,305XX,Web,Closed with monetary relief,Yes,No,1474177,0,1,1,0,0,0,0


In [21]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
Issue : 95
Company : 3276
State : 62
ZIP code : 25962
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2


In [22]:
for col in ['ZIP code','Company']:
    cd_train.drop([col],axis=1,inplace=True)
    cd_test.drop([col],axis=1,inplace=True)

In [23]:
cd_train['Consumer disputed?']=np.where(cd_train['Consumer disputed?']=="Yes",1,0)

In [24]:
#cd_test['Consumer disputed?']=cd_train['Consumer disputed?']

In [25]:
cd_train['Consumer disputed?'].value_counts()

Consumer disputed?
0    376990
1    101431
Name: count, dtype: int64

In [26]:
k=cd_train['Issue'].value_counts()
for val in k.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['Issue']==val,1,0)
    cd_test[varname]=np.where(cd_test['Issue']==val,1,0)
del cd_train['Issue']
del cd_test['Issue']

In [27]:
for col in cd_train.select_dtypes(['object']).columns:
    print(col,':',cd_train[col].nunique())

Product : 12
State : 62
Submitted via : 6
Company response to consumer : 7
Timely response? : 2


In [28]:
k=cd_train['State'].value_counts()
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_train[varname]=np.where(cd_train['State']==val,1,0)
    cd_test[varname]=np.where(cd_test['State']==val,1,0)
del cd_train['State']
del cd_test['State']

In [29]:
cd_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478421 entries, 0 to 478420
Data columns (total 38 columns):
 #   Column                                          Non-Null Count   Dtype 
---  ------                                          --------------   ----- 
 0   Product                                         478421 non-null  object
 1   Submitted via                                   478421 non-null  object
 2   Company response to consumer                    478421 non-null  object
 3   Timely response?                                478421 non-null  object
 4   Consumer disputed?                              478421 non-null  int32 
 5   Complaint ID                                    478421 non-null  int64 
 6   day_diff                                        478421 non-null  int64 
 7   Sub_product_isNan                               478421 non-null  int32 
 8   Sub_issue_isNan                                 478421 non-null  int32 
 9   Consumer_complaint_narrative_isNan   

In [30]:
cd_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119606 entries, 0 to 119605
Data columns (total 37 columns):
 #   Column                                          Non-Null Count   Dtype 
---  ------                                          --------------   ----- 
 0   Product                                         119606 non-null  object
 1   Submitted via                                   119605 non-null  object
 2   Company response to consumer                    119606 non-null  object
 3   Timely response?                                119606 non-null  object
 4   Complaint ID                                    119606 non-null  int64 
 5   day_diff                                        119606 non-null  int64 
 6   Sub_product_isNan                               119606 non-null  int32 
 7   Sub_issue_isNan                                 119606 non-null  int32 
 8   Consumer_complaint_narrative_isNan              119606 non-null  int32 
 9   Company_public_response_isNan        

In [31]:
#cd_test.drop('Consumer disputed?',axis=1)

In [32]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_train[col],prefix=col,drop_first=True)
    cd_train=pd.concat([temp,cd_train],axis=1)
    cd_train.drop([col],axis=1,inplace=True)
    
    temp=pd.get_dummies(cd_test[col],prefix=col,drop_first=True)
    cd_test=pd.concat([temp,cd_test],axis=1)
    cd_test.drop([col],axis=1,inplace=True)


In [33]:
cd_train.shape

(478421, 57)

In [34]:
cd_test.shape

(119606, 56)

In [35]:
x=cd_train.drop(['Consumer disputed?','Complaint ID'],axis=1)
y=cd_train['Consumer disputed?']


In [36]:
#x_test=cd_test.drop('Consumer disputed?',axis=1,inplace=True)

In [37]:
##########new code###########################

In [38]:
from sklearn.model_selection import train_test_split

In [39]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [40]:
y_test.shape

(95685,)

In [41]:
from sklearn.preprocessing import StandardScaler

In [42]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

In [43]:
#!!pip install imbalanced-learn

In [44]:
#pip install --upgrade scikit-learn

In [45]:
from imblearn.over_sampling import SMOTE

In [46]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

In [47]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [48]:
params={'penalty':['l1','l2'],'class_weight':['balanced',None],
        'C':np.linspace(0.00001,1,100)}


In [49]:
model=LogisticRegression(C=0.0001, solver='liblinear', max_iter=10)

In [50]:
rs=RandomizedSearchCV(model,param_distributions=params,n_iter=10,
                      scoring='roc_auc',cv=10,n_jobs=-1,verbose=20)

In [51]:
rs.fit(X_train_resampled,y_train_resampled)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


RandomizedSearchCV(cv=10,
                   estimator=LogisticRegression(C=0.0001, max_iter=10,
                                                solver='liblinear'),
                   n_jobs=-1,
                   param_distributions={'C': array([1.00000000e-05, 1.01109091e-02, 2.02118182e-02, 3.03127273e-02,
       4.04136364e-02, 5.05145455e-02, 6.06154545e-02, 7.07163636e-02,
       8.08172727e-02, 9.09181818e-02, 1.01019091e-01, 1.11120000e-01,
       1.21220909e-01, 1.31321818e-01, 1.4...
       8.08082727e-01, 8.18183636e-01, 8.28284545e-01, 8.38385455e-01,
       8.48486364e-01, 8.58587273e-01, 8.68688182e-01, 8.78789091e-01,
       8.88890000e-01, 8.98990909e-01, 9.09091818e-01, 9.19192727e-01,
       9.29293636e-01, 9.39394545e-01, 9.49495455e-01, 9.59596364e-01,
       9.69697273e-01, 9.79798182e-01, 9.89899091e-01, 1.00000000e+00]),
                                        'class_weight': ['balanced', None],
                                        'penalty': ['l1', 'l2']},
                   scoring='roc_auc', verbose=20)

In [52]:
train_score=rs.predict_proba(x)[:,1]

In [53]:
train_score=rs.predict_proba(X_train_resampled)[:,1]

In [54]:
train_score

array([0.20214628, 0.59720658, 0.58094327, ..., 0.8147007 , 0.70002084,
       0.54772458])

In [55]:
y_pred=rs.predict_proba(X_test_scaled)

In [56]:
y_pred

array([[0.11638573, 0.88361427],
       [0.33396042, 0.66603958],
       [0.00527376, 0.99472624],
       ...,
       [0.0136382 , 0.9863618 ],
       [0.0437354 , 0.9562646 ],
       [0.00264074, 0.99735926]])

In [57]:
x_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95685 entries, 419668 to 286964
Data columns (total 55 columns):
 #   Column                                                        Non-Null Count  Dtype
---  ------                                                        --------------  -----
 0   Timely response?_Yes                                          95685 non-null  bool 
 1   Company response to consumer_Closed with explanation          95685 non-null  bool 
 2   Company response to consumer_Closed with monetary relief      95685 non-null  bool 
 3   Company response to consumer_Closed with non-monetary relief  95685 non-null  bool 
 4   Company response to consumer_Closed with relief               95685 non-null  bool 
 5   Company response to consumer_Closed without relief            95685 non-null  bool 
 6   Company response to consumer_Untimely response                95685 non-null  bool 
 7   Submitted via_Fax                                             95685 non-null  bool 


In [58]:
from sklearn.metrics import roc_auc_score

In [59]:
# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_train_resampled, train_score)

In [60]:
roc_auc

0.7693773400416584

In [61]:
# Calculate the ROC AUC score
roc_auc = roc_auc_score(y_test,  y_pred[:, 1])

In [62]:
roc_auc

0.5649486351898608

In [63]:
#cd_test.drop(['Consumer disputed?'],axis=1,inplace=True)

In [65]:
cd_test.drop(['Complaint ID'],axis=1,inplace=True)

KeyError: "['Complaint ID'] not found in axis"

In [66]:
test_score=rs.predict_proba(cd_test)[:,1]

In [67]:
test_score

array([0.52788832, 0.44833294, 0.47835995, ..., 0.21444698, 0.24938708,
       0.46162392])

In [68]:
cutoffs=np.linspace(0.01,0.99,99)
cutoffs

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [69]:
real=y_train

In [70]:
from sklearn.metrics import fbeta_score

In [71]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)
    
    
    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()

    P=TP+FN
    N=TN+FP


    KS=(TP/P)-(FP/N)

    KS_all.append(KS)

 # try out what cutoffs you get when you use F_beta scores with different values of betas [0.5 , 5]
 # beta < 1 : you will get cutoff , which is high ( favours precision)
 # beta > 1 : you will get cutoff , which is low (favours precision ) 

ValueError: operands could not be broadcast together with shapes (602978,) (382736,) 

In [73]:
list(zip(cutoffs,KS_all))

[]

In [74]:
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff # gi

ValueError: max() arg is an empty sequence

In [75]:
test_classes=(test_score>mycutoff).astype(int)

NameError: name 'mycutoff' is not defined

In [205]:
test_classes

array([1, 0, 0, ..., 0, 0, 1])

In [210]:
pd.DataFrame(test_score).to_csv("mysubmission_p1_new_roc.csv",index=False)
submission1=pd.DataFrame(list(zip(cd_train['Complaint ID'],["Yes" if value == 1 else "No" for value in list(test_classes)])),
                       columns=['Complaint ID','Consumer disputed?'])

In [213]:
submission1.to_csv('NidhiSharma_submission_Project1.csv',index=False)

In [214]:
submission1

,Complaint ID,Consumer disputed?
0,856103,Yes
1,1034666,No
2,756363,No
3,1474177,No
4,1132572,Yes
...,...,...
119601,800351,No
119602,1926642,Yes
119603,1271555,No
119604,1980041,No


In [ ]:
#submission1=pd.DataFrame(list(zip(cd_train['Complaint ID'],list(test_classes))),
                    #   columns=['Complaint ID','Consumer disputed?'])

In [ ]:
##########################################################################################

In [ ]:
# Analyzing the outcome variable
cd_train['Consumer disputed?'].value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
params={'class_weight':['balanced',None],
 'penalty':['l1','l2'],
 'C':np.linspace(0.1,1000,10)}

In [ ]:
model=LogisticRegression(fit_intercept=True)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_search=GridSearchCV(model,param_grid=params,cv=10,scoring="roc_auc")

In [ ]:
grid_search.fit(x,y) #

In [ ]:
grid_search.best_estimator_

In [ ]:
logr=grid_search.best_estimator_


In [ ]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


In [ ]:
report(grid_search.cv_results_,5)


In [ ]:
logr.fit(x,y)

In [ ]:
train_score=logr.predict_proba(x)[:,1] # we can get the probabilities using predict_proba


In [ ]:
train_score


In [ ]:
#test_score=logr.predict_proba(cd_test)[:,1]



In [ ]:
#test_score

In [ ]:
#submission=pd.DataFrame(test_score).to_csv("mysubmission.csv",index=False)


In [ ]:
#pd.DataFrame(test_score).to_csv("mysubmission.csv",index=False)
#submission1=pd.DataFrame(list(zip(cd_train['Complaint ID'],list(test_score))),
                       columns=['Complaint ID','Consumer disputed?'])


In [ ]:
#submission.head(4)

In [ ]:
#submission1.to_csv('sample_submission.csv',index=False)

This submission will get you auc score of approx 0.50, slightly less than whats required for passing the course. You'll have to make changes 

In [147]:
cutoffs=np.linspace(0.01,0.99,99)
cutoffs


array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99])

In [134]:
#train_score=logr.predict_proba(x_train)[:,1] # the predicted response variable
#values
real=y # the actual response variable values
print(logr.classes_) # In order to find the probability of which column is for outcome 1 and which for outcome 0
real

NameError: name 'logr' is not defined

In [149]:
real=y

In [150]:
real

0         0
1         0
2         0
3         0
4         0
         ..
478416    0
478417    0
478418    0
478419    0
478420    0
Name: Consumer disputed?, Length: 478421, dtype: int32

In [135]:
from sklearn.metrics import fbeta_score

In [151]:
KS_all=[]

for cutoff in cutoffs:
    
    predicted=(train_score>cutoff).astype(int)
    
    
    TP=((predicted==1) & (real==1)).sum()
    TN=((predicted==0) & (real==0)).sum()
    FP=((predicted==1) & (real==0)).sum()
    FN=((predicted==0) & (real==1)).sum()

    P=TP+FN
    N=TN+FP


    KS=(TP/P)-(FP/N)

    KS_all.append(KS)

 # try out what cutoffs you get when you use F_beta scores with different values of betas [0.5 , 5]
 # beta < 1 : you will get cutoff , which is high ( favours precision)
 # beta > 1 : you will get cutoff , which is low (favours precision ) 

In [152]:
list(zip(cutoffs,KS_all))

[(0.01, 0.04223116830767959),
 (0.02, 0.04223116830767959),
 (0.03, 0.04223116830767959),
 (0.04, 0.04223116830767959),
 (0.05, 0.04223116830767959),
 (0.060000000000000005, 0.04223116830767959),
 (0.06999999999999999, 0.04223116830767959),
 (0.08, 0.04223116830767959),
 (0.09, 0.04223116830767959),
 (0.09999999999999999, 0.04223116830767959),
 (0.11, 0.04223116830767959),
 (0.12, 0.04223116830767959),
 (0.13, 0.04223116830767959),
 (0.14, 0.04225769421022341),
 (0.15000000000000002, 0.04305523717819809),
 (0.16, 0.04733636007573505),
 (0.17, 0.05871496547996674),
 (0.18000000000000002, 0.0721048280184331),
 (0.19, 0.08861678939091538),
 (0.2, 0.09922561078105657),
 (0.21000000000000002, 0.08452960080817498),
 (0.22, 0.0663633112800458),
 (0.23, 0.02767257161598731),
 (0.24000000000000002, 0.0009484698903647563),
 (0.25, 0.00043672288209270493),
 (0.26, 0.00043672288209270493),
 (0.27, 0.00043672288209270493),
 (0.28, 0.00043672288209270493),
 (0.29000000000000004, 0.000436722882092704

In [153]:
mycutoff=cutoffs[KS_all==max(KS_all)][0]
mycutoff # gi

0.2

In [ ]:
#cd_test['Complaint ID']=cd_train['Complaint ID']

In [ ]:
#cd_test.drop(['Complaint ID'],axis=1,inplace=True)


In [ ]:
#cd_test.drop(['Consumer disputed?'],axis=1,inplace=True)

In [ ]:
test_score=logr.predict_proba(cd_test)[:,1]
#test_score

In [ ]:
test_score

In [ ]:
#cd_test[real]=real
#cd_test.drop[-1]

In [156]:
test_classes=(test_score>mycutoff).astype(int)

In [157]:
test_classes

array([1, 0, 0, ..., 0, 0, 1])

In [ ]:
cd_test['Consumer disputed?']=cd_train['Consumer disputed?']

In [ ]:
real_array =cd_test['Consumer disputed?'].values

In [ ]:
real_array

In [ ]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(real, (train_score))

print("AUC Score:", auc_score)

In [ ]:
from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(real_array, (test_score))

print("AUC Score:", auc_score)

In [ ]:
submission1=pd.DataFrame(list(zip(cd_train['Complaint ID'],list(test_classes))),
                       columns=['Complaint ID','Consumer disputed?'])

In [ ]:
submission1.to_csv('sample_submission_new.csv',index=False)